*******

# Running VNTRseek on any dataset

_________
Tools used: 
`samtools` and `bedtools`
  
Need:  
1. wgs dataset (sample genome)
    - in either FASTQ, BAM, or CRAM formats
2. GRCh38 reference genome
    - without alts (chr1-22,XY,M) on SCC: /project/vntrseek/share/GRCh38/GRCh38.fa
    - with alts (correct if using data from 1000 Genomes project)
        - [reference genome file](https://github.com/igsr/1000Genomes_data_indexes/blob/master/data_collections/gambian_genome_variation_project/README_gambian_genome_variation_project.md)   
            \*must be the one used to map original FASTQ files
            - download via
                `lftp`
                `pget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa` 

To load:   
`module load samtools`  
`module load bedtools`

NOTE: any terminal commands starred\* means that they should go inside a script
 - Script how-to at end of guide
_________

#### Downloading Files

 - Make a new directory in /projectnb/vntrseek for yourself (ex: sfiller)
 - Within it create a folder with title of wgs data you are mapping (ex: NYGC_na18545)
 - command into NYGC_na18545 directory
 - type `lftp`
 - to download files:
     - FASTQ 
         - `pget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR323/007/ERR3239357/ERR3239357_1.fastq.gz`
         - `pget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR323/007/ERR3239357/ERR3239357_2.fastq.gz`
     - CRAM
         - `pget ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR323/ERR3239357/NA18545.final.cram`
 - Now unzip each file
     - `gunzip file.name`

#### Input Datasets

VNTRseek accepts 2 file types:
 1. FASTQ
 2. BAM

Will input datasets be ...  
 a) **complete** (unfiltered)  
 b) **restricted** (filtered)

**A)** If running complete dataset (not filtering): 
 - use FASTQ
     - place fastq files (2) in their own directory
 - use BAM 
     - place BAM file in its own directory
     - index the BAM file so it is also in the same directory
         - `samtools index file_name.bam`
         

**B)** If running a restricted dataset (filtered) - need to get into BAM format

 - if starting from FASTQ
    1. Map 2 paired-end fastq files to reference genome - output: BAM file
        

HOW: 
________________________
- Copy map_one_file.sh script from /projectnb/vntrseek/scripts/bwa_alignment/map_one_file.sh into your personal directory NYGC_na18545, modify
    - add flags:   
      `set e` stops the execution of the script if an error occurs  
      `#$ -P vntrseek` to denote which project and to use that project's specific resources  
      `-j y` to join output and error stream files into one   
      `#$ -m bea` to send an email when job begins/ends/aborts  
      `#$ -M my@email.com` overwrite default email and send to YOUR email included  
      `-l h_rt=200:00:00`  had to set time limit to longer than 100 hours - takes a while  
      `-pe omp 8` request number of threads  
    - comment out the module load samtools and module load bwa (were causing errors)    
- In NYGC_na18545 directory, `module load samtools` and `module load bwa`  
- Then submit job:  
     - `qsub -N na18545 -o na18545.log map_one_file.sh /projectnb/vntrseek/sfiller/NYGC_na18545/ERR3239357_1.fastq /projectnb/vntrseek/sfiller/NYGC_na18545/ERR3239357_2.fastq na18545 8`  
     
         `-N` job name  
         `-o` output of job in following file name     
         `map_one_file.sh` script used, followed by arguments for that script  
             - arg 1: /path/to/read1.fq  
             - arg 2:  /path/to/read2.fq  
             - arg 3: name_of_bam_file  
             - arg 4: number of processors to use
--------------------

- if starting from CRAM 
    1. convert CRAM to BAM:     
        \*script needed\*  `projectnb/vntrseek/sfiller/scripts/cram_to_bam.sh`  
        
        `samtools view -b  -T GRCh38_full_analysis_set_plus_decoy_hla.fa -o NA18545.final.bam NA18545.final.cram`  
        - `-b` for bam output
        - `-o` to create an output file 
        - `-T` for reference file
        - first arg: [reference genome file](https://github.com/igsr/1000Genomes_data_indexes/blob/master/data_collections/gambian_genome_variation_project/README_gambian_genome_variation_project.md)   
            \*must be the one used to map original FASTQ files
            - download via
                `lftp`
                `pget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa`  
        - second arg: name of output bam (will be placed in current directory)
        - third arg: name of input cram (must exist)  
          
            

CHECK if number of reads are same between FASTQ files and BAM files: 
_____________

 - Number of reads in fastq file    
     `wc -l practice_na18545/ERR3239357_1.fastq`  
     Output: 1579052668
       
     Output is number of lines. Four lines per read in a fastq file, but there are also 2 files (paired reads), so *2/4 the output, or just divide by two:  



In [8]:
reads = 1579052668/2
print(reads)

789526334.0


 - Number of reads in bam file  
   
 Use `samtools view -c` options - give you number of ALIGNMENTS (can be larger than the actual number of reads)  
   
> To include only reads with certain bitwise flags - add up the bit values of the flags you want to INCLUDE:  
            `samtools view -c -f <bitValue> file.bam`  
              
 > To exclude reads with certain bitwise flags - add up bit values of the flags you want to EXCLUDE:  
            `samtools view -c -F <bitValue> file.bam`
  
I want to exclude reads that were secondary alignments (256) *AND* any supplementary reads (2048) (in order to get my total number of distinct reads)  
256+2048 = 2034 (leaves only primary alignments or unmapped reads)

In [13]:
#samtools view -c -F 256 na18545.bam #same output when using bam made from fastq files (bwa_bam)
samtools view -c -F 2034 na18545.bam #same as above for bwa_bam, different for db_bam

Output: 789526334  
  
Both numbers record the same number of reads, move on.

#### If using a restricted read dataset (if not skip to "Run VNTRseek")

##### Use samtools and bedtools to extract primary alignment reads that map with overlap to TR regions

1. Extract primary reads with Samtools from bam file and redirect into new bam file

In [ ]:
samtools view -b -F 2038 na18545.bam > primary_na18545.bam

- `-F 2038` excludes:   
  4 - unmapped reads   
  256 -secondary alignments   
  2048-supplementary alignments (necessary for BAM that used GRCh38 with alts)  
- `-b` output is in BAM format

2. Take only those primary reads that overlap with TR loci using Bedtools\*

Bedtools Intersect process: compare A line by line to B, to see if any reads from A overlap with regions from B, if there are, return all overlaps of A with B only once, output into a bam file
  
[Bedtools intersect](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html)  
`bedtools intersect -a A.bed -b B.bed [options]`  
 

 I created `extract_overlaps.sh`

In [ ]:
#!/bin/bash  

#$ -P vntrseek  
#$ -j y  
#$ -l h_rt=100:00:00  
#$ -pe omp 8  
#$ -V  
#$ -m e  
#$ -M sfiller@friars.providence.edu  

module load samtools  
module load bedtools

# first parameter is A file, bam file
# second parameter is B file, bed file
# third parameter is output file

bedtools intersect -u -abam $1 -b $2 > $3    

- \$1 is A file, use `abam` for bam file input
    - A file  - larger file (loaded a line at a time)
- \$2 is B file, smaller file (loaded to memory) - TR ref set BED file
- `-u` Write original A entry once if any overlaps found in B. In other words, just report the fact at least one overlap was found in B.  
 \*do NOT use -ubam (truncates the output bam file)
- \$3 output file name (BAM output)

Must format bed file containing TR regions, start with TR_Human_Refset_228486.bed:  

1. Make sure 4 column (original) bed is tab delimited: 

In [ ]:
cat TR_Human_Refset_228486.bed| perl -lane 'print "$F[0]\t$F[1]\t$F[2]\t$F[3]"' > TabDL
M_TR_Human_Refset_228486.bed

 2. Need to comment out header from TabDLM file (did via text editor)

3. Then compare with bedtools intersect to extract overlaps using a script, extract_overlaps.sh

4. Run script via submitting a job

In [ ]:
qsub -N extractOverlaps /projectnb/vntrseek/sfiller/scripts/extract_overlaps.sh primary_na18545.bam TabDLM_TR_Refset.bed TR_overlaps_primaryreads_na18545.bam 

#### VARIATIONS of restricted read dataset
1. Add unmapped reads
2. Add unmapped reads + primary alignments mapping to alternate chromosome regions  
    - can only do for BAM converted from db CRAM

1. **FIRST VARIATION**: Use samtools to extract unmapped reads and add them to primary TR overlaps

Get unmapped reads

In [ ]:
samtools view -b -f 4 na18545.bam > unmapped.bam

 - `-b` for bam output
 - `-f 4`to include only reads that are unmapped 

Sort both the unmapped reads bam + primary reads that overlap TRs bam files (must do prior to merging)

 - Use samtools sort\*  

In [ ]:
samtools sort ex.bam -o ex.sorted.bam

 - Use `sortbam.sh` script  
     - takes one argument: full path of file to sort, must exclude .bam ending  
     - creates an output file with original name with .sorted.bam ending

Combine both sorted bam files using samtools merge

In [ ]:
samtools merge primaryoverlaps_unmapped.bam TR_overlaps_primaryreads_na18545.sorted.bam unmapped.sorted.bam

First arg: output name  
Second arg: first sorted file  
Third arg: second sorted file

2. **SECOND VARIATION**: extract and add reads overlapping with alt chromosomes to unmapped reads and primary TR overlaps

\* Only possible for bam converted from CRAM because it was mapped using reference genome with alt chromosomes

EXTRACT primary reads that mapped to alternate chromosomes

 - Use bedtools intersect method - to extract only reads that aligned to alternative chromosomes(and any other parts of the reference genome that are not contained in the GRCh38.fa reference)
     - Take index file for GRCh38 with alts reference genome and take only alternate chromosomes (everything after line 25) and turn into bed file (take only first and second columns - chr name and chr length, add column of zeroes in between as start of each chromosome)

In [ ]:
awk 'FNR>25 {print $1 "\t0\t" $2}' GRCh38_full_analysis_set_plus_decoy_hla.fa.fai>alt_chr_intervals.bed

 - Now use this BED file with bedtools intersect to extract only primary reads that overlap with alt chr regions - output should be a bam of all these
     - use extract_overlaps.sh scripts and submit a job (from /projectnb/vntrseek/sfiller directory)
        - \$1 is BAM containing primary reads
        - \$2 is BED containing alt chr regions
        - \$3 path of output file with output file name (BAM output)

In [ ]:
qsub -N extractOverlaps /projectnb/vntrseek/sfiller/scripts/extract_overlaps.sh /projectnb/vntrseek/sfiller/snake_workflow/primary_reads/NA18545.final.bam alt_chr_intervals.bed /projectnb/vntrseek/sfiller/snake_workflow/primary_reads/altchr_overlaps_NA18545.final.bam 

Follow steps in Variation 1 to sort output BAM then merge this file (containing primary reads overlapping with alt chr regions) with the bam file containing primary reads overlapping with TR regions and unmapped reads (primary_TR_overlaps+unmapped reads)

**Organization of files** / Snakemake method

Can also use [Snakemake](https://snakemake.readthedocs.io/en/stable/tutorial/short.html) to streamline efforts
 - I used to go from original BAM > primary reads> sort> merge with unmapped reads
 - create a Snakefile

Using Snakemake requires folders for each step (see /projectnb/sfiller/NYGC_HG00096 for a concise example)
 - folders must be thoughtfully organized ^, file name stays the same, ex: NA18545.bam  

If not using Snakemake organization method
 - files should be thoughtfully named

#### Run VNTRseek

If running FASTQ files make sure that both files are together in a directory (nothing else in directory)

If running BAM file, need to index before, store BAM and BAM index files together in a directory

 - Use `inxbam.sh` script to index   
     - $1 need to type full path of file to index including .bam ending  
     - uses `samtools index` command

If using restricted read datasets, run all three variations:

>1. primary reads overlapping with TR regions (primary_TR_overlaps)
2. primary_TR_overlaps + unmapped
3. primary_TR_overlaps + unmapped + primar_altchr_overlaps

Run VNTRseek by submitting a job, see Assignment 3 notebook for further explanation.

In [ ]:
qsub -N nameOfJob vntr.v10.sh directoryNAMEofFiles startStep readLength
#qsub -N primaryTRoverlapsVNTR vntr.v10.sh primaryoverlaps_VNTRtest 0 150

- First parameter after vntr.v10.sh is the directory containing the file(s) to be run and where output will go
- Second parameter is step of VNTRseek you want to start on
- Third parameter is length of each read in the file

#### Record RUN Stats in spreadsheet

After receiving email results of job completion need to record all stats (especially CPU time and wallclock time)

Use `run_comparison_results.xlsx` (on github) as outline

>\* look at Runtimes.xlsx format if using multiple runs of each kind of restricted read dataset (I used this file only to average the run times in my limited analysis)

#### Compare output vcf files

Also record numreads, numreadTRs, numVNTRs, numTRsWithSupport on output `.span2.vcf` files
 - \* .span2.vcf files contain only VNTRs found
 - \* allwithsupport.vcf files contain all TRs and VNTRs found

Using `comparevcf.py script` (on github) to compare vntrs found

*How to Use*  
 - Use a text editor/IDE/notebook (I used VSCode)   
 - Once in the editor, connect to the remote server/SSH using BU login and PW  
 - Use anaconda `module load miniconda`
 - Create an environment using anaconda, `conda create --name vcfpy python=3`    
     - \*Called my environment 'vcfpy' (not to be confused with the library)  
 - Now activate the environment `conda activate vcfpy`  
 - Install libraries needed  
    - `pip install vcfpy`
    - `pip install pysam`
 - After setting up the environment (above), **regularly load via**:   
     - `module load miniconda`
     - `conda activate vcfpy`
     - To run script
         1. Make sure in the directory of the script
         2. Type python version you would like to use, the script name, then the complete paths to the two vcf files you would like to compare (make sure to use .span2.vcf files to compare VNTRs only)  
           `python3 comparevcf.py vcf/path/one vcf/path/two`

Record differences in run_comparison_results.xlsx

To run PlotVNTRresults.ipynb (github) to visualize all results found, save spreadsheet as .csv file (in same directory as the notebook)

#### OPTIONAL: Move resulting .db file from each run of VNTRseek to orca (to view in VNTRview)

First gzip each db file

In [ ]:
!gzip file.db

Then secure copy (scp) the file over to the orca server, will be prompted for password

In [ ]:
!scp file.db.gz sfiller@orca.bu.edu:/home/sfiller/(fileName optional)

Once .db.gz file is processed and ready to view on VNTRview, can delete copy in orca directory

Once viewable on VNTRview, go to export, right click VCF ALL VNTRS and copy link

In [ ]:
!wget <copied link> #to download

To view file and check num_VNTRS found

In [1]:
!more file.name

file.name: No such file or directory


QSUB stats:  
- Wallclock time: difference between end time and start time  
- CPU: cpu time usage in seconds (hours:minutes:seconds)  
- maxvmem: the maximum virtual memory (size in bytes) that has been used during the CPU runtime  

#### How to write a bash script - submit to queue as a noninteractive batch job via `qsub`
`#!/bin/bash` called the shebang   
`#$ -P vntrseek` which project's resources you will be using  
`#$ -j y`  j for join error and output stream files into one, y for yes  
`#$ -l h_rt=100:00:00`  time limit  
`#$ -pe omp 8` number of processors to use   
`#$ -V` all current environment variables should be exported to the batch job  
`#$ -m bea` instances when you want to be emailed: Beginning, End, or Abort  
`#$ -M desired@email.com` email to send job status to   

\*Useful [GUIDE](https://www.bu.edu/tech/files/2020/01/2020_spr-Tutorial-Intermediate-Usage-of-Shared-Compute-Cluster-SCC.pdf) for submitting jobs through SCC